In [29]:
from unittest import result
import streamlit as st
import pymysql
import pandas as pd
from malicious_news import crawl_malicious_news
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import parsing_html
from parsing_html import getHtml, to_csv, get_csv
import predict_module
from predict_module import predict_from_csv




def compare_benign_malicious_chart(train_csv_path, features):
    try:
        # === 1. TrainData에서 benign 평균 구하기 ===
        df_train = pd.read_csv(train_csv_path)
        df_train.columns = df_train.columns.str.replace(r"html_num_tags\('", "", regex=True).str.replace(r"'\)", "", regex=True)
        df_train = df_train.dropna(subset=features + ['repu'])

        scaler = MinMaxScaler()
        df_train_scaled_values = scaler.fit_transform(df_train[features])
        df_train_scaled = pd.DataFrame(df_train_scaled_values, columns=features)
        df_train_scaled['repu'] = df_train['repu'].values

        mean_benign = df_train_scaled[df_train_scaled['repu'] == 'benign'][features].mean().values
        mean_malicious = df_train_scaled[df_train_scaled['repu'] == 'malicious'][features].mean().values
        print(mean_benign)
        df1 = df_train_scaled[df_train_scaled['repu'] == 'benign'][features]
        df2 = df_train_scaled[df_train_scaled['repu'] == 'malicious'][features]
        #print(f'mean_benign: {df1}')
        print(mean_malicious)
        #print(f'mean_malicious: {df2}')

        # === 3. 레이더 차트 준비 ===
        labels = features
        num_vars = len(labels)
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]

        mean_malicious = np.concatenate((mean_malicious, [mean_malicious[0]]))
        mean_benign = np.concatenate((mean_benign, [mean_benign[0]]))

        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
        ax.plot(angles, mean_malicious, label='악성 평균 (malicious)', color='red')
        ax.plot(angles, mean_benign, label='정상 평균 (benign)', color='blue')
        ax.fill(angles, mean_malicious, color='red', alpha=0.25)
        ax.fill(angles, mean_benign, color='blue', alpha=0.15)
        ax.set_yticks([0.2, 0.4])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels)
        ax.set_title('악성 평균 vs 정상 평균 Feature 비교', size=15)
        ax.legend(loc='upper right')
        st.pyplot(fig)
    except Exception as e:
        st.error(f"차트 생성 중 오류 발생: {e}")




# 파일 경로
train_csv = "BE/TrainDataAll.csv"
user_csv = "extract_feature.csv"

dftd = pd.read_csv("BE/TrainDataAll.csv")

# Feature 목록
features = [                        
                    "url_entropy",
                    "url_path_len", "url_filename_len", "url_longest_dom_token_len",
                    "url_average_dom_token_len", "url_domain_len", "url_hostname_len", 
                    "url_port",
                    "script","div"
                
                ]
   
compare_benign_malicious_chart(train_csv, features)


[0.40550671 0.04849013 0.01961446 0.13206426 0.13983562 0.14810344
 0.14808548 0.00041288 0.06677151 0.04593012]
[0.45380252 0.0273512  0.00891281 0.24761769 0.23902525 0.226311
 0.22623021 0.0018582  0.01978568 0.01633242]
